In [41]:
#!unzip Test.zip

In [42]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [43]:
suffix = '.pgm'
kaggle_files = os.listdir('Test')
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [44]:
len(pgm_kaggle_files)

8000

In [45]:
kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = 'Test/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
  kaggle_patches.append(image)

100%|██████████| 8000/8000 [00:01<00:00, 4930.23it/s]


In [46]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

100%|██████████| 8000/8000 [00:00<00:00, 3688516.21it/s]


In [47]:
import joblib

In [48]:
# Aquí debe utilizar su modelo
model = joblib.load('models/knn_model.joblib')

In [49]:
from skimage import feature

# Aquí debe procesar las imagenes de pgm_kaggle_files
X_kag = []

suffix = '.pgm'
dir = os.listdir('Test/')
pgm_files = [filename for filename in dir if filename.endswith(suffix)]

for filename in tqdm(pgm_files):
    path = 'Test/' + filename
    with open (path, 'rb') as pgmf:
        image = plt.imread(pgmf)
        X_kag.append(image)
        
X = np.array([feature.hog(im) for im in tqdm(X_kag, desc='Construyendo X') ])


Construyendo X: 100%|██████████| 8000/8000 [00:03<00:00, 2397.21it/s]


In [56]:
from sklearn.decomposition import PCA 

pca = PCA(n_components=500)
X_pca_kag = pca.fit(X)
#x_transform = pca.transform(X)

# Predicción
y_kag = model.predict(X_pca_kag)

ValueError: Expected 2D array, got scalar array instead:
array=PCA(n_components=500).
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [ ]:
kaggle_hat.head()

In [18]:
from datetime import datetime

fecha_hora = datetime.now().strftime("%Y-%m-%d %H:%M")

print(fecha_hora)

2025-06-08 21:50


In [ ]:
# Guardamos en un .csv para subir a kaggle
submission_name = f'knn_{fecha_hora}'
kaggle_hat.to_csv(submission_name + '.csv', index=False)